# OpenAI Parameters

# Overview
When making requests to OpenAI models, several parameters can be used to control the behavior and output of the model. \
Understanding these parameters helps in fine-tuning the responses to meet specific requirements, whether for generating text, answering questions, or any other use case.

For more detailed examples, refer to the official documentation [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
SEED = 123

# Parameter: max_tokens
**Description**: The maximum number of tokens to generate in the completion. \
**Default**: 16 \
**Example**: max_tokens=50

The token count of your prompt plus max_tokens can't exceed the model's context length. \
Most models have a context length of 2048 tokens (except for the newest models, which support 4096). Please refer to documentation.

In [2]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=max_tokens
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [16, 32, 60, 100]
for penalty in penalties:
    print(f"Max Tokens: {penalty}\n")
    print(call_openai_with_max_tokens(penalty))
    print("\n" + "-"*80 + "\n")

Max Tokens: 16

The best pet really depends on various factors, including your lifestyle, living situation,

--------------------------------------------------------------------------------

Max Tokens: 32

The best pet really depends on individual preferences and lifestyles. Here are some considerations for various popular pets:

1. **Dogs**: If you want a loyal and

--------------------------------------------------------------------------------

Max Tokens: 60

The "best" pet can vary greatly depending on individual preferences, lifestyle, living conditions, and what a person is looking for in a pet. Here are some popular choices:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active people or families. They require regular

--------------------------------------------------------------------------------

Max Tokens: 100

The "best" pet can vary greatly depending on an individual's lifestyle, preferences, and living situation. Here are some popular optio

# Parameter: temperature

**Description**: Controls the randomness of the output. Lower values make the output more deterministic, while higher values increase randomness. \
**Value Range**: 0 to 1 \
**Default Value**: 1 \
**Example**: temperature=0.7

Higher values means the model will take more risks. \
Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.

---
**NOTE**: We generally recommend altering this or top_p but not both.


In [3]:
def call_openai(num_times, prompt, temperature=0.75, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = temperature
            )
        else:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    temperature = temperature
            )
        print(response.choices[0].message.content)

In [4]:
# Without seed and temperature, the response is different each time
call_openai(10, 'The best pet is a ')

The best pet really depends on your personal preferences, lifestyle, and living situation. Here are a few popular options and what they might offer:

1. **Dogs**: Known for their loyalty and companionship. They can be great for active individuals or families, as they often need exercise and enjoy spending time
The best pet really depends on your personal preferences, lifestyle, and living situation. Here are some common options, each with its own benefits:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families who have time for walking, training, and playing.
The best pet really depends on your lifestyle, preferences, and what you're looking for in a companion. Here are a few popular options:

1. **Dogs**: Great for companionship, loyalty, and active households. They require regular exercise and attention.
2. **Cats**: Independent and low-maint
The best pet really depends on your lifestyle, preferences, and living situation. Here are 

In [5]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, 'The best pet is a ', temperature = 0, use_seed=True)

The best pet really depends on your lifestyle, preferences, and what you're looking for in a companion. Here are a few popular options, each with their own benefits:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families. They require regular exercise and
The best pet really depends on your lifestyle, preferences, and what you're looking for in a companion. Here are a few popular options, each with their own benefits:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families. They require regular exercise and
The best pet really depends on your lifestyle, preferences, and what you're looking for in a companion. Here are a few popular options, each with their own benefits:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families. They require regular exercise and
The best pet really depends on your lifestyle, preferences, and what you're 

# Parameter: n
**Description**: Specifies the number of completions to generate for each prompt. \
**Default Value**: 1 \
**Example**: n = 3 

---
**Note**: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

In [6]:
response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages = [{"role":"system", "content":"You are a helpful assistant."},
                        {"role":"user","content": "The best pet is a "}],
                max_tokens=60,
                n=2
        )

for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 The best pet for you depends on various factors, including your lifestyle, living situation, personal preferences, and the amount of time and resources you can dedicate to a pet. Here are some considerations for popular pets:

1. **Dogs**: Great companions for active people who enjoy outdoor activities and can commit
1 The best pet really depends on your personal preferences, lifestyle, and living situation. Here are a few popular options based on different needs and preferences:

1. **Dogs**: Often considered loyal and great companions. They can be very social and require regular exercise and attention. They come in a variety of


# Parameter: presence_penalty
**Description**: Penalizes new tokens based on whether they appear in the text so far, encouraging the model to use new tokens. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: presence_penalty=0.5

In [7]:
def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=60,
                    presence_penalty=presence_penalty
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")

Presence Penalty: 0

Choosing the best pet depends on your lifestyle, preferences, and needs. Here are a few options based on different criteria:

1. **Dogs**: Great for companionship and active lifestyles. They're loyal and require regular exercise.
2. **Cats**: More independent than dogs, usually require less maintenance,

--------------------------------------------------------------------------------

Presence Penalty: 0.5

The best pet can vary depending on an individual's lifestyle, preferences, and needs. Here are a few considerations for different types of pets:

1. **Dogs:** Great companions, highly social, and vary widely in size, temperament, and activity level. Ideal for those who enjoy outdoor activities and have time

--------------------------------------------------------------------------------

Presence Penalty: 1.0

The "best" pet depends on various factors, including your lifestyle, living situation, and personal preferences. Different animals can make great pets fo

# Parameter: frequency_penalty
**Description**: Penalizes new tokens based on their existing frequency in the text so far, reducing the likelihood of repeating the same line verbatim. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: frequency_penalty=0.5

#### Use cases to explore
1. **Compare Responses** \
Generate multiple completions to compare and choose the best response for your use case.

2. **Increase Diversity** \
Use multiple completions to get a variety of responses, which is useful in creative applications.

3. **Enhance Robustness** \
Generate multiple responses to ensure consistency and accuracy across different completions.

#### Best Practices
1. **Optimize Prompt Length** \
Keep your prompts concise but informative to ensure the model has enough context.

2. **Adjust Temperature and Top_p** \
Use these parameters to balance between deterministic and creative responses.

3. **Monitor Token Usage** \
Be mindful of the max_tokens parameter to manage costs and response length.

4. **Use Stopping Sequences** \
Define stopping sequences to control where the model should stop generating text, ensuring the output is within the desired context.

5. **Generate Multiple Completions** \
Use the n parameter to generate multiple completions and select the best one for your needs.